In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# 데이터 불러오기
movie_data = pd.read_csv('archive/movies.csv')
rating_data = pd.read_csv('archive/ratings.csv')

# 영화와 사용자의 평점 테이블 만들기
movie_user_rating = rating_data.pivot_table('rating', index='movieId', columns='userId').fillna(0)

# 코사인 유사도 계산
item_based_cola_sim = cosine_similarity(movie_user_rating)
item_based_cola_sim_df = pd.DataFrame(data=item_based_cola_sim, index=movie_user_rating.index, columns=movie_user_rating.index)

# 'Leaving Las Vegas (1995)' 영화의 movieId 찾기
movie_id = movie_data[movie_data['title'] == 'Leaving Las Vegas (1995)']['movieId'].values[0]

# 'Leaving Las Vegas (1995)' 영화와 가장 유사한 영화 찾기 (자신 제외)
most_similar_movie_id = item_based_cola_sim_df[movie_id].sort_values(ascending=False).index[1]  # 첫 번째는 자신이므로 두 번째로 유사한 영화 선택

# 가장 유사한 영화 제목 출력
most_similar_movie_title = movie_data[movie_data['movieId'] == most_similar_movie_id]['title'].values[0]
most_similar_movie_title

'Dead Man Walking (1995)'

In [2]:
movie_user_rating.index[item_based_cola_sim[movie_user_rating.index.get_loc(input("영화 제목: "))].argsort()[-2:-7:-1]]

KeyboardInterrupt: Interrupted by user

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# 데이터 불러오기
movie_data = pd.read_csv('archive/movies.csv')
rating_data = pd.read_csv('archive/ratings.csv')

# 영화 제목과 평점을 함께 포함하도록 데이터 병합
data = pd.merge(rating_data, movie_data, on='movieId')

# 사용자-아이템 행렬 생성
movie_user_rating = data.pivot_table(index='userId', columns='title', values='rating')

# 결측값을 0으로 채움
movie_user_rating.fillna(0, inplace=True)

# 코사인 유사도 행렬 계산
item_based_cola_sim = cosine_similarity(movie_user_rating.T)

# 유사도 행렬을 DataFrame으로 변환하여 다루기 쉽게 함
item_based_cola_sim_df = pd.DataFrame(item_based_cola_sim, index=movie_user_rating.columns, columns=movie_user_rating.columns)

# 사용자에게 영화 제목을 입력받아 유사한 영화 5편 출력
movie_title = input("영화 제목: ")

# 영화 제목이 데이터셋에 있는지 확인
if movie_title in movie_user_rating.columns:
    # 영화 제목의 인덱스를 찾음
    movie_index = movie_user_rating.columns.get_loc(movie_title)
    
    # 유사도 점수를 추출
    similarities = item_based_cola_sim_df.iloc[movie_index]
    
    # 유사도 점수를 기준으로 상위 5개의 영화 인덱스를 찾음 (자기 자신 제외)
    top_similar_indices = similarities.nlargest(6).index[1:6]
    
    # 유사한 영화 제목들
    similar_movies = movie_user_rating.columns[top_similar_indices]
    print("유사한 영화 5편:", similar_movies)
else:
    print(f"영화 '{movie_title}'가 데이터셋에 없습니다.")
